# Streaming prediction-error filters

Prediction-error filters (PEFs) are essential in seismic deconvolution and other geophysical estimation problems. We show that non-stationary multidimensional PEFs can be computed in a "streaming" manner, where the filter gets updated incrementally by accepting one new data point at a time. The computational cost of estimating a streaming PEF reduces to the cost of a single convolution. In other words, the cost of PEF design while filtering equals the cost of applying the filter. Moreover, the non-linear operation of finding and applying a streaming PEF is invertible at a similar cost, which enables a fast approach to missing data interpolation.

# Introduction

Prediction-error filters (PEFs) play an essential role in geophysical data analysis. They can be used directly in different forms of seismic deconvolution (Webster, 1978; Robinson & Osman, 1996). More importantly, PEFs are a practical approximation for covariance operators used in geophysical estimation problems (Claerbout, 2014). As such, they provide a way to absorb and characterize statistical patterns in a given dataset (Claerbout & Brown, 1999).

Because most natural patterns are non-stationary, extending the classic stationary formulation to non-stationarity is vital. This extension can be done either by "patching" or breaking data into overlapping windows or by a smoothly non-stationary estimation with regularization (Crawley et al., 1999; Curry, 2003). Shaping regularization provides a particularly effective method for constraining the filter variability (Fomel, 2009; Liu & Fomel, 2011; Liu et al., 2012). Both approaches have an additional computational expense, particularly in storing variable filter coefficients (Ruan et al., 2015).

This paper presents an efficient approach to computing and applying non-stationary PEFs with no excessive storage requirements. Using an adaptive-filtering approach (Widrow \& Stearns, 1985; Haykin, 2002), We show that a non-stationary PEF can be updated on the fly by accepting one data point at a time. The cost of computing and applying such a PEF reduces to the cost of a single convolution. In other words, the cost of PEF design while filtering equals the cost of applying the filter. Moreover, the non-linear operation of finding and applying a PEF has an exact inverse, which finds an immediate application in missing data interpolation problems. We extend the
filter from 1-D to multiple dimensions using the helix transform (Claerbout, 1998) and show its application to simple benchmark problems.

## Theory       

The basic equation defining a prediction-error filter is auto-regression. For a given data stream $\mathbf{d}$ and a one-dimensional PEF $\mathbf{a}$, the auto-regression equation takes the form

$$\left[\begin{array}{ccccc} d_{n+1} & d_n & d_{n-1} & \cdots &
                                                   d_{n+1-k} \end{array}\right]\,
\left[\begin{array}{l} 1 \\ a_1 \\ a_2 \\ \cdots \\ a_k \end{array}\right]
\approx 0\;,$$

where it is assumed that $n \ge k$. Equation 1 represents the convolution of the data with the prediction-error filter.

Suppose the filter gets updated with each new data point
$d_{n+1}$. The additional constraint we can impose to control
the variability of the filter is that the new filter $a$ stays close
to the previous filter $\bar{a}$.

The two conditions can be combined into one overdetermined linear system

$$\left[\begin{array}{cccc} d_n & d_{n-1} & \cdots & d_{n+1-k} \\
      \lambda & 0 & & 0 \\
      0 & \lambda & & 0 \\
    \cdots & \cdots & & \cdots \\
      0 & 0 & \cdots & \lambda
    \end{array}\right]\,
  \left[\begin{array}{l} a_1 \\ a_2 \\ \cdots \\ a_k \end{array}\right] \approx
  \left[\begin{array}{l} - d_{n+1} \\
      \lambda\,\bar{a}_1 \\
      \lambda\,\bar{a}_2 \\
     \cdots \\
      \lambda\,\bar{a}_k
    \end{array}\right]\;,$$

where $\lambda$ is the parameter that controls how much we allow 
$a$ to deviate from $\bar{a}$. In a shortened block-matrix notation, we can rewrite the linear system of equations as

$$\left[\begin{array}{c} \mathbf{d}^T \\ \lambda\,\mathbf{I} \end{array}\right]\,\mathbf{a} \approx
  \left[\begin{array}{l} - d_{n+1} \\
      \lambda\,\bar{\mathbf{a}} 
    \end{array}\right]\;,$$
    
where

$$\mathbf{d} = \left[\begin{array}{l} d_n \\ d_{n-1} \\ \cdots \\ d_{n+1-k} \end{array}\right]\;, \quad
\mathbf{a} = \left[\begin{array}{l} a_1 \\ a_2 \\ \cdots \\ a_k \end{array}\right]\;,$$

and $\mathbf{I}$ is the identity matrix. 

The least-squares solution of the overdetermined system is

$$\mathbf{a} = \left(\mathbf{d}\,\mathbf{d}^T + \lambda^2\,\mathbf{I}\right)^{-1}\,
  \left(-d_{n+1}\,\mathbf{d} + \lambda^2\,\bar{\mathbf{a}}\right)\;.$$

Next, we can use the Sherman-Morrison formula from linear algebra
(Hager, 1989) to transform the inverse matrix as follows:

$$\left(\mathbf{d}\,\mathbf{d}^T + \lambda^2\,\mathbf{I}\right)^{-1} =
  \frac{1}{\lambda^2}\,\left(\mathbf{I} - \frac{\mathbf{d}\,\mathbf{d}^T}{\lambda^2+\mathbf{d}^T\,\mathbf{d}}\right)\;.$$

Substituting the Sherman-Morrison equation and doing
algebraic simplifications lead to the final result:

$$\begin{array}{rcl}\mathbf{a} & = & \displaystyle \frac{1}{\lambda^2}\,\left(\mathbf{I} -
                   \frac{\mathbf{d}\,\mathbf{d}^T}{\lambda^2+\mathbf{d}^T\,\mathbf{d}}\right)\,\left(-d_{n+1}\,\mathbf{d}
                   + \lambda^2\,\bar{\mathbf{a}}\right) \\
& = &
\displaystyle \bar{\mathbf{a}} -
  \left(\frac{d_{n+1}+\mathbf{d}^T\,\bar{\mathbf{a}}}{\lambda^2+\mathbf{d}^T\,\mathbf{d}}\right)\,\mathbf{d}\;.\end{array}$$
  
The Sherman-Morrison technique is a well-known approach in adaptive filtering by recursive least-squares (Haykin, 2002).
  
This equation shows that the filter is updated by subtracting
a scaled version of the data. The scale is proportional to the
convolution residual computed using the previous version of the
filter. Updating the filter requires only elementary algebraic operations (vector dot products) and no iterations. Moreover, computing the dot product
$\mathbf{d}^T\,\mathbf{d}$ in a "streaming" fashion requires at most
two multiplications:

$$\mathbf{d}^T\,\mathbf{d} = \bar{\mathbf{d}}^T\,\bar{\mathbf{d}} +
d_n^2 - d_{n-k}^2\;,$$

where $k$ is number of points in $\mathbf{a}$ and $\mathbf{d}$.

### Inversion

The streaming residual $r_{n+1}$ from the left-hand side of
equation 1 can be expressed as

$$r_{n+1} = d_{n+1} + \mathbf{d}^T\,\mathbf{a}$$

or, substituting equation for $\mathbf{a}$,

$$\begin{array}{rcl}r_{n+1} & = & \displaystyle d_{n+1} + \mathbf{d}^T\,\left[\bar{\mathbf{a}} -
              \left(\frac{d_{n+1}+\mathbf{d}^T\,\bar{\mathbf{a}}}{\lambda^2+\mathbf{d}^T\,\mathbf{d}}\right)\,\mathbf{d}\right]
  \\
\nonumber
& = & \displaystyle \left(d_{n+1} +
      \mathbf{d}^T\,\bar{\mathbf{a}}\right)\,\left(1-\frac{\mathbf{d}^T\,\mathbf{d}}{\lambda^2+\mathbf{d}^T\,\mathbf{d}}\right)
  \\
& = & \displaystyle \frac{\lambda^2\,\left(d_{n+1} +
  \mathbf{d}^T\,\bar{\mathbf{a}}\right)}{\lambda^2+\mathbf{d}^T\,\mathbf{d}}\;.\end{array}$$


The equation for $r_{n+1}$ can be directly inverted to reconstruct~$d_{n+1}$ as follows:

$$d_{n+1} = r_{n+1}\,\left(1+\frac{\mathbf{d}^T\,\mathbf{d}}{\lambda^2}\right) - \mathbf{d}^T\,\bar{\mathbf{a}}\;.$$

We see that streaming time-variable deconvolution is invertible: both the input data and the time-varying filter can be reconstructed from the streaming residual. Note that because of the division by $\lambda^2$, the inverse operation may become numerically unstable for small $\lambda$.

In [ ]:
n1=400

# exponentially decaying sinusiods with different frequencies
inp = zeros(Float32,n1)
for j in n1÷15:n1÷5:n1
    wave = zeros(Float32,n1)
    for x in j:399
        y = (x-j)/400
        wave[x] = exp(-y*15)*sin(y*0.95*j)
    end
    inp += wave
end

res = similar(inp);
bak = similar(inp);

In [ ]:
import Pkg; Pkg.add("Plots")

In [ ]:
using Plots

function stems(data, label, color) 
    "plot data using stems"
    plt=plot(zeros(Float32, n1), label=:none, color=:black)
    plot!(plt, data, line=:stem,
          label=label, color=color, legend=:outerleft, 
          xlim=[0.5, n1+0.5], border=:none)   
    return plt
end

In [ ]:
plot_input = stems(inp,"input",:blue)

In [ ]:
function stream!(inv::Bool, d::Vector{T}, r::Vector{T}, na::Int, λ::Real) where T <: Real
    a = zeros(T, na) # streaming PEF
    dd = da = zero(T) # d (dot) d, d (dot) a
    for ia in 1:na
        if (inv) 
            d[ia] = r[ia]
        else 
            r[ia] = d[ia]
        end
        dd += d[ia]*d[ia]
    end
    for i1 in na+1:n1
        if (inv) # from r to d
            rn = r[i1] / λ 
            dn = rn * (λ + dd) - da
            d[i1] = dn
        else     # from d to r
            dn = d[i1]
            rn = (dn + da) / (λ + dd)
            r[i1] = λ * rn
        end
        # update PEF
        for ia in 1:na; a[ia] -= rn * d[i1-ia]; end
        # update dd and da
        dd += dn*dn - d[i1-na] * d[i1-na]   
        da = dn * a[1]
        for ia in 2:na; da += a[ia] * d[i1-ia+1]; end
    end
end

In [ ]:
stream!(false, inp, res, 2, 0.1)
plot_decon = stems(res, "decon", :green);

In [ ]:
stream!(true, bak, res, 2, 0.1)
plot_inverse = stems(bak,"inverse",:purple);

In [ ]:
plot(plot_input, plot_decon, plot_inverse, layout=(3, 1))

In [ ]:
savefig("stream.pdf")

### Multiple dimensions

In order to extend the filter from 1-D to multiple dimensions, we follow the
helix transform of Claerbout (1998). The change is minimal:
the multidimensional data is arranged in a 1D stream on a helix, and
the definition of the data vector $\mathbf{d}$ changes to

$$\mathbf{d} = \left[\begin{array}{l} d_{n+1-l_1} \\ d_{n+1-l_2} \\ \cdots \\ d_{n+1-l_k} \end{array}\right]$$

where $l_1, l_2, \ldots, l_k$ represent lags of the helical filter. The computational cost and other benefits of streaming PEFs remain the same.

One caveat is that, in multiple dimensions, we may want the
nonstationary filter to change smoothly along the helix and other dimensions. As shown below, this goal can be accomplished with only a minor change to the algorithm. Suppose that $\bar{\mathbf{a}}_1$ is the previous filter in the first (helical) dimension, and $\bar{\mathbf{a}}_2$ is the previous filter in the second dimension. We can keep the newly estimated streaming PEF close to both
filters by changing the matrix equation to

$$\left[\begin{array}{c} \mathbf{d}^T \\ \lambda_1\,\mathbf{I} \\  \lambda_2\,\mathbf{I}  \end{array}\right]\,\mathbf{a} \approx
  \left[\begin{array}{l} - d_{n+1} \\
      \lambda_1\,\bar{\mathbf{a}}_1 \\ \lambda_2\,\bar{\mathbf{a}}_2
    \end{array}\right]\;,$$

where $\lambda_1$ and $\lambda_2$ control the filter variability in the two directions. The least-squares solution of this system is

$$\mathbf{a} = \left(\mathbf{d}\,\mathbf{d}^T + \lambda^2\,\mathbf{I}\right)^{-1}\,
  \left(-d_{n+1}\,\mathbf{d} + \lambda_1^2\,\bar{\mathbf{a}}_1 +
    \lambda_1^2\,\bar{\mathbf{a}}_2 \right)\;,$$

where $\lambda^2 = \lambda_1^2+\lambda_2^2$. The new equation
is equivalent to the previous equation with the simple substitution

$$\bar{\mathbf{a}} = \displaystyle \frac{\lambda_1^2\,\bar{\mathbf{a}}_1 +
    \lambda_2^2\,\bar{\mathbf{a}}_2}{\lambda_1^2+\lambda_2^2}\;.$$

Thus, the only change in the algorithm is an increase in the storage to keep track of both $\bar{\mathbf{a}}_1$ and $\bar{\mathbf{a}}_2$. The extension to more dimensions is analogous.

### Cost comparison

Table 1 compares the computational cost of different approaches to computing prediction-error filters. The cost of streaming PEF is minimal and reduces to the cost of a single convolution. Streaming PEF can capture the input data's non-stationary character without storing multiple copies of the filter.

Moreover, all streaming computations are local, which makes them
particularly suitable for modern hardware accelerators such as
GPGPU (Sanders & Kandrot, 2010) or Intel Xeon Phi (Jeffers & Reinders, 2013).

| Method | Storage | Cost |
|:-------|:--------|:-----|
| Stationary PEF | $O(N_a)$ | $O(N_a^2\,N_d)$ |
| Non-stationary PEF | $O(N_a\,N_d)$ | $ O(N_a^2\,N_d)$ |
| Streaming PEF | $O(N_a)$ | $O(N_a\,N_d)$ |

## Missing data interpolation

The existence of the exact inversion for the application of a streaming PEF allows for a straightforward approach to missing data interpolation. When reading data in a streaming fashion, every time we meet a missing data point $d_{n+1}$, we can replace its value by a
value computed from the residual. The residual value $r_{n+1}$ in this case can be set to zero or to a random number (white noise) with the expected variance of the residual. In the latter case, it is possible to generate multiple equiprobable distributions for the interpolation
result (Clapp, 2000).

In [ ]:
inp2 = deepcopy(inp)
known = ones(Bool,n1)

# Cut holes in the data and create a mask
for hole in (55, 153, 246, 301, 376)
    inp2[hole:hole+20] .= 0
    known[hole:hole+20] .= false
end

In [ ]:
function stream_missing!(d::Vector{T}, k::Vector{Bool}, na::Int, λ::Real) where T <: Real
    a = zeros(T, na) # streaming PEF
    da = zero(T) # d (dot) a
    dd = zero(T) # d (dot) d
    for ia in 1:na
        dd += d[ia]*d[ia]
    end
    for i1 in na+1:n1
        if (k[i1]) # from d to r
            dn = d[i1]
            rn = (dn + da) / (λ + dd)
        else       # assume r=0
            dn = - da
            rn = zero(T)
            d[i1] = dn
        end
        # update PEF
        for ia in 1:na
            a[ia] -= rn * d[i1-ia]
        end
        # update dd and da
        dd += dn*dn - d[i1-na] * d[i1-na]   
        da = dn * a[1]
        for ia in 2:na
            da += a[ia] * d[i1-ia+1]
        end
    end
end

In [ ]:
plot_ideal = stems(inp,"ideal ",:blue);
plot_hole = stems(inp2,"input ",:green);

In [ ]:
miss = deepcopy(inp2)
stream_missing!(miss,known,2,0.05)

In [ ]:
plot_interp = stems(miss,"filled",:purple);

In [ ]:
plot(plot_ideal, plot_hole, plot_interp, layout=(3, 1))

In [ ]:
savefig("mstream.pdf")

A simple 1D interpolation example using data from Figure 1 is shown in Figure 2. As evident from the figure, such interpolation, while capable of picking the non-stationary data pattern, remains one-sided and may create discontinuities at the other side of the data gap. We can rerun the streaming PEF using the opposite directions and stack the results to help with this issue.

In [ ]:
import Pkg; Pkg.add("ZipFile")

In [ ]:
using ZipFile

# download data from a public server
download("https://zenodo.org/api/records/11099632/files-archive", "files.zip")
# unzip the archive file
r = ZipFile.Reader("files.zip")

In [ ]:
# make a dictionary of files for easy access
patterns = Dict{String, IO}()
for file in r.files
    name = splitext(file.name)[1]
    patterns[name] = file
end

In [ ]:
# download "wood" pattern
wood = Array{Float32}(undef, 128, 128) # single-precision array
read!(patterns["wood"], wood)

In [ ]:
heatmap(wood,axis=nothing,legend=:none,color=:grays)

In [ ]:
function punch_hole(data::Matrix{T}) where T <: Real
    # make an elliptical hole
    n1, n2 = size(data)
    hole = similar(data)
    mask = zeros(Bool, n1, n2)
    for i2 in 1:n2, i1 in 1:n1
        x = (i1-1)/n1 - 0.5
        y = (i2-1)/n2 - 0.3
        u =  x + y
        v = (x - y)/2
        if (u*u + v*v < 0.15)
            hole[i1,i2] = zero(T)
        else
            hole[i1,i2] = data[i1,i2]
            mask[i1,i2] = true
        end
    end
    return hole, mask
end

whole, wmask = punch_hole(wood);

In [ ]:
function helix(lag::Vector{Tuple{T, T}}, ci::CartesianIndices) where T <: Integer
    "convert filter lags to helix lags for a given grid"
    # middle of the grid
    mid = CartesianIndex(Tuple(last(ci)) .÷ 2)
    # helix index of middle
    hmid = LinearIndices(ci)[mid]
    # from Cartesian shift to helix shift
    return LinearIndices(ci)[map(x -> CartesianIndex(x) + mid, lag)] .- hmid
end

In [ ]:
import Random

function stream_missing_helix!(d, k, 
                               lag::Vector{Tuple{I, I}},
                               λ::Real, std=0, seed=1) where I <: Integer
    "Fill missing data in multiple dimensions using streaming PEF on a helix"
    n1, na = length(d), length(lag)
    hlag = helix(lag, CartesianIndices(d))
    maxlag = maximum(hlag)
    T = eltype(d)
    a = zeros(T, na) # streaming PEF
    da = zero(T) # d (dot) a
    dd = zero(T) # d (dot) d
    for ia in 1:na
        dd += d[maxlag+1-hlag[ia]]^2
    end
    Random.seed!(seed)
    for i1 in maxlag+1:n1
        if (k[i1])
            dn = d[i1]
            rn = (dn + da) / (λ + dd)
        else # assume r_n is random
            rn = std * randn() / λ
            dn = rn * (λ + dd) - da
            d[i1] = dn
        end
        # update PEF
        for ia in 1:na
            a[ia] -= rn * d[i1-hlag[ia]]
        end
        # update dd and da
        dd += dn * dn - d[i1-maxlag] * d[i1-maxlag]   
        da = dn * a[1]
        for ia in 2:na
            da += a[ia] * d[i1+1-hlag[ia]]
        end
    end
end

In [ ]:
# 11 x 11 PEF
lag=[(x,0) for x in 1:5]
for k in 1:10
    lag = vcat(lag,[(x,k) for x in -5:5])
end

In [ ]:
function fill_hole(forward::Bool, hole, mask, pad::Integer, noise=0, seed=1)
    if forward
        holepad = hcat(zeros(Float32, size(hole, 1), pad), hole)
        maskpad = hcat(zeros(Bool, size(hole, 1), pad), mask)
        stream_missing_helix!(holepad, maskpad, lag, 1e6, noise, seed)
        return holepad[:,pad+1:end]
    else
        rhole = reverse(hole)
        rmask = reverse(mask)
        holepad = hcat(zeros(Float32, size(rhole, 1), pad), rhole)
        maskpad = hcat(zeros(Bool, size(rhole, 1), pad), rmask)
        stream_missing_helix!(holepad, maskpad, lag, 1e6, noise, seed+1)
        return reverse(holepad[:,pad+1:end])
    end
end

In [ ]:
filled1 = fill_hole(true, whole, wmask, 20);
filled2 = fill_hole(false, whole, wmask, 20);

In [ ]:
plot2(data, title) = heatmap(data, axis=nothing, yflip=:true, clim=(-137, 137),
                             legend=:none, color=:grays, title=title)

In [ ]:
p1 = plot2(filled1, "(a) Filled 1")
p2 = plot2(filled2, "(a) Filled 2")
p3 = plot2(filled1 + filled2 - whole, "(c) Filled 1+2")
plot(p1, p2, p3, layout=(1, 3))

In [ ]:
savefig("interp.pdf")

In [ ]:
filled = fill_hole(true,  whole, wmask, 20, 2) + 
         fill_hole(false, whole, wmask, 20, 2) - whole

p1 = plot2(wood, "(a) Ideal") 
p2 = plot2(whole, "(b) Gapped")
p3 = plot2(filled, "(c) Filled")
plot(p1, p2, p3, layout=(1, 3))

In [ ]:
savefig("wood-hole.pdf")

Figure 3 shows a 2D missing data interpolation test using a synthetic pattern from  Claerbout & Brown (1999). The input data for this test is shown in Figure 4b. Interpolation proceeds by running the streaming PEF twice in the forward and backward directions (Figures 3a and 3b) and averaging the interpolation results (Figure 3c.) The cost of such procedure is the cost of two convolutions, as opposed to multiple iterations required in the conventional approach to missing data interpolation with PEFs  (Naghizadeh & Sacchi, 2010; Liu & Fomel, 2011; Claerbout, 2014). A better interpolation result in Figure 4c is achieved by filling the residual inside the gap with small-variance white noise instead of zeros while reconstructing
the data from the residual. 

In [ ]:
p = Array{Plots.Plot}(undef,3)
for k in 1:3
    filled = fill_hole(true,  whole, wmask, 20, 2, k) + 
             fill_hole(false, whole, wmask, 20, 2, k+3) - whole
    p[k] = plot2(filled, "Realization $k") 
end
plot(p[1], p[2], p[3], layout=(1, 3))

In [ ]:
savefig("realiz.pdf")

The random residual approach can generate multiple equiprobable realizations for the missing data interpolation problem in the spirit of geostatistical stochastic simulations (Clapp, 2000). Figure 5 shows three realizations created using different seeds for the pseudorandom number generator. 

In [ ]:
# "herring" pattern
herr = Array{Float32}(undef, 128, 128) # single-precision array
# read data
read!(patterns["herr"], herr)
# make a hole
hhole, hmask = punch_hole(herr);

In [ ]:
filled = fill_hole(true,  hhole, hmask, 20, 6) + 
         fill_hole(false, hhole, hmask, 20, 6) - hhole

p1 = plot2(herr, "(a) Ideal") 
p2 = plot2(hhole, "(b) Gapped")
p3 = plot2(filled, "(c) Filled")
plot(p1, p2, p3, layout=(1, 3))

In [ ]:
savefig("herr-hole.pdf")

Figure 6 shows a similar interpolation test applied to 2D data with a non-stationary pattern. Although streaming PEF fails to achieve a perfect reconstruction in this case, it manages to capture most of the pattern and hide the location of the gap.

In [ ]:
# "seismic" pattern
seis = Array{Float32}(undef, 250, 125) # single-precision array
# read data
read!(patterns["seis"], seis)

In [ ]:
using Statistics

# normalize
m = mean(seis)
seis .-= m
scale = std(wood)/std(seis)
seis *= scale

# make a hole
shole, smask = punch_hole(seis);

In [ ]:
filled = fill_hole(true,  shole, smask, 20, 0.7) + 
         fill_hole(false, shole, smask, 20, 0.7) - shole

p1 = plot2(seis, "(a) Ideal") 
p2 = plot2(shole, "(b) Gapped")
p3 = plot2(filled, "(c) Filled")
plot(p1, p2, p3, layout=(1, 3))

In [ ]:
savefig("WGstack-hole.pdf")

Figure 7 shows a similar test applied to 2D seismic data. The pattern made by reflection events with variable slopes is well captured.

In [ ]:
function stream_helix!(inv::Bool, d, r, lag::Vector{Tuple{I, I}}, λ::Real) where I <: Integer
    n1, na = length(d), length(lag)
    hlag = helix(lag, CartesianIndices(d))
    maxlag = maximum(hlag)
    T = eltype(d)
    a = zeros(T, na) # streaming PEF
    for i1 in 1:maxlag
        if (inv) 
            d[i1] = r[i1]
        else 
            r[i1] = d[i1]
        end
    end
    da = zero(T) # d (dot) a
    dd = zero(T) # d (dot) d
    for ia in 1:na      
        dd += d[maxlag+1-hlag[ia]]^2
    end
    for i1 in maxlag+1:n1
        if (inv) 
            rn = r[i1] / λ 
            dn = rn * (λ + dd) - da
            d[i1] = dn
        else 
            dn = d[i1]
            rn = (dn + da) / (λ + dd)
            r[i1] = λ * rn
        end
        # update PEF
        for ia in 1:na
            a[ia] -= rn * d[i1-hlag[ia]]
        end
        # update dd and da
        dd += dn * dn - d[i1-maxlag] * d[i1-maxlag]    
        da = dn * a[1]
        for ia in 2:na
            da += a[ia] * d[i1+1-hlag[ia]]
        end
    end
end

In [ ]:
# apply helix filter
pad = hcat(zeros(Float32, size(seis, 1), 20), seis)
res= similar(pad)
stream_helix!(false, pad, res, lag, 1e6) # pad -> res
stream_helix!(true,  pad, res, lag, 1e6) # pad <- res

In [ ]:
p1 = plot2(seis, "(a) Input") 
p2 = plot2(20*res[:,21:end], "(b) Residual (x 20)")
p3 = plot2(pad[:,21:end], "(c) Inverse")
plot(p1, p2, p3, layout=(1, 3))

In this case, removing dominant reflection events by PEF filtering leaves behind weaker hyperbolic diffraction events (Figure 8). 

## Discussion

A streaming application, where the adaptive filter is updated one data point at a time, is appropriate in a continuous stream of large amounts of data, such as in passive monitoring of carbon storage (Alumbaugh et al., 2024). In more typical scenarios, where the data are immediately accessible, more accurate results can be achieved with other forms of regularization, such as regularizing adaptive filter coefficients with smoothing shaping operators (Fomel, 2009; Liu \& Fomel, 2011; Liu et al., 2012). For greater efficiency, a hybrid regularization strategy can be developed. Such strategies are discussed by Geng et al. (2024), who also extend the streaming approach to other applications of seismic attributes.

In applications such as missing data reconstruction, the streaming approach represents an extreme point of the accuracy-efficiency trade-off. To achieve better accuracy, some of the efficiency can be sacrificed at the expense of performing extra iterations. The presented approach is also limited to situations of missing values in regularly sampled data and will need to be modified for situations of irregular data.

In multidimensional applications, the helical boundary conditions allow for easy invertibility (Claerbout, 1998) but are not always appropriate. In this case, extra accuracy can also be bought at the cost of sacrificing some of the efficiency.

We hope that providing reproducible benchmarks with this paper will invite other researchers to make direct comparisons with more advanced methods. Such comparisons are beyond the scope of this paper because they would require a different codebase.

## Conclusions

We have presented an efficient approach to computing and applying non-stationary prediction-error filters (PEFs). Instead of storing multiple copies of varying filters, the streaming approach stores only one copy and updates the filter on the fly with every new data point. The cost of this procedure is equivalent to the cost of a single convolution and does not require multiple iterations. Moreover, the non-linear operation of estimating and applying a streaming PEF has an exact inverse, which becomes helpful in missing data interpolation
problems. A streaming approach to missing data interpolation does not require iterations and can be accomplished effectively at the cost of two convolutions. We anticipate many possible applications of the proposed technique in geophysical estimation problems.

## References

Alumbaugh, D.L., Correa, J., Jordan, P., Petras, B., Chundur, S. and Abriel, W., 2024. An assessment of the role of geophysics in future US geologic carbon storage projects. The Leading Edge, 43, 72-83.

Bezanson, J., A. Edelman, S. Karpinski, and V. B. Shah, 2017, Julia: A fresh approach to numerical computing: SIAM Review, 59, 65-98.

Claerbout, J., 1998, Multidimensional recursive filters via a helix: Geophysics, 63, 1532–1541.

Claerbout, J., and M. Brown, 1999, Two-dimensional textures and prediction-error filters: 61st Mtg., Eur. Assn. Geosci. Eng., Session:1009.

Claerbout, J. F., 2014, Geophysical image estimation by example: Environmental soundings image enhancement: Lulu. (http://sep.stanford.edu/sep/prof/).

Clapp, R., 2000, Multiple realizations using standard inversion techniques, in SEP-105: Stanford Exploration Project, 67–78.

Crawley, S., J. Claerbout, and R. Clapp, 1999, Interpolation with smoothly non-stationary prediction-error filters: 69th Annual International Meeting, SEG, Expanded Abstracts, 1154–1157.

Curry, W., 2003, Interpolation of irregularly sampled data with nonstationary, multi- scale prediction-error filters: 73th Annual International Meeting, SEG, Expanded Abstracts, 1913–1916.

Fomel, S., 2009, Adaptive multiple subtraction using regularized nonstationary regression: Geophysics, 74, V25–V33.

Geng, Z., Fomel, S., Liu, Y., Wang, Q., Zheng, Z. and Chen, Y., 2024. Streaming seismic attributes. Geophysics, 89, A7-A10.

Granger, B.E. and F. P\'{e}rez, F., 2021, Jupyter: Thinking and storytelling with code and data: Computing in Science \& Engineering, 23, 7-14.

Hager, W. W., 1989, Updating the inverse of a matrix: SIAM Review, 31, 221–239. Jeffers, J., and J. Reinders, 2013, Intel Xeon Phi coprocessor high-performance programming: Morgan Kaufmann.

Haykin, S., 2002. Adaptive Filter Theory. Prentice-Hall.

Liu, G., X. C. J. Du, and K. Wu, 2012, Random noise attenuation using f-x regularized nonstationary autoregression: Geophysics, 77, V61–V69.

Liu, Y., and S. Fomel, 2011, Seismic data interpolation beyond aliasing using regularized nonstationary autoregression: Geophysics, 76, V69–V77.

Naghizadeh, M., and M. D. Sacchi, 2010, Robust reconstruction of aliased data using autoregressive spectral estimates: Geophysical Prospecting, 58, 1049–1062.

Robinson, E. A., and O. M. Osman, eds., 1996, Deconvolution 2: Soc. of Expl. Geophys.

Ruan, K., J. Jennings, E. Biondi, R. G. Clapp, S. A. Levin, and J. Claerbout, 2015, Industrial scale high-performance adaptive filtering with PEF applications, in SEP-160: Stanford Exploration Project, 177–188.

Sanders, J., and E. Kandrot, 2010, CUDA by example: An introduction to General- Purpose GPU programming: Addison-Wesley Professional.

Webster, G. M., ed., 1978, Deconvolution: Soc. of Expl. Geophys.

Widrow, B., and S.D. Stearns, Adaptive Signal Processing: Prentice Hall, 1985.